## Homework 2: Joins

In this assignment, we'll use a variety of data sources to quantify the existence of food deserts in LA County. The assignment is *loosely* based on the food pantry example from class. It focuses on honing your skills in processing and joining data together, particularly spatially.

A quick note: It's great to look at your variables, dataframes, etc. while you are exploring the data. But **please comment out those exploratory lines of code before submitting**. It makes it hard for us to find your solutions.

### Policy on ChatGPT and similar LLMs

ChatGPT and similar tools can be useful in trying to figure out the syntax for a complex command, or to understand and fix an error. However, if you jump directly to ChatGPT for an answer, that means you aren't learning anything. 

Therefore, you may you use ChatGPT or a similar tool for this homework if you:
- clearly acknowledge when and how you used the tool (e.g., "I got a syntax error when trying to join the two dataframes, and asked ChatGPT for help")
- attempt the question on your own first, and try to fix any errors. Include your original attempts in your submission (you can comment out the lines that don't work)
- briefly discuss what you learned from ChatGPT (e.g., "I learned that function definitions need a colon (:) after the def statement."). Don't just uncritically accept ChatGPT's answer - learn from it!

If you have any questions, talk to the instructor before you submit.

*Please help me grade* by observing the following:
 
* Do not rename this notebook (that messes up the autograder)
* Do not include large sections of output (that makes it hard to find your code). For example, use `df.head()` to show the first few rows, rather than printing an entire dataframe. The same goes for printing long strings.

Your homework repository includes a zipped csv file `snap.zip` of SNAP-authorized retailers, as of April 2025. [It was downloaded from USDA.](https://www.fns.usda.gov/snap/retailer-locator) To my knowledge, this is the most comprehensive list of grocery outlets in the US.

Load it into a pandas dataframe called `snapDf`. Keep only the observations that are in Los Angeles County. 

*Hint:* pandas can read a `.zip` file directly...no need to unzip first.

In [ ]:
# your code here
snapDf = 999  # replace with your DataFrame
    
### BEGIN SOLUTION

import pandas as pd
snapDf = pd.read_csv('snap.zip')
snapDf = snapDf[snapDf.County=='LOS ANGELES']
### END SOLUTION

In [ ]:
# Autograding tests - do not edit
print(snapDf.columns)
print(snapDf.County.unique())
print(len(snapDf))
assert all(snapDf.columns == ['X', 'Y', 'Record_ID', 'Store_Name', 'Store_Street_Address', 'Additonal_Address',
       'City', 'State', 'Zip_Code', 'Zip4', 'County', 'Store_Type', 'Latitude', 'Longitude', 'Incentive_Program',
       'Grantee_Name', 'ObjectId'])
assert len(snapDf)==8477

If you look at the store names, many of the places that accept SNAP benefits are liquor stores and gas stations. These might have an important role where no other food is available, but are likely to have a limited range of food, particularly fresh produce. 

Most of these stores are labeled as `Convenience Store`. (That will also capture many other types of store, e.g. 7-11, but let's not worry about that for now.)

Create a histogram of the `Store_Type` field. (pandas has a histogram function; you could also use the `histplot` function in the `seaborn` library.)

In [ ]:
# your code here

### BEGIN SOLUTION

# you could use the built in pandas hist() function too, but seaborn makes nicer plots
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots()
sns.histplot(data=snapDf, x="Store_Type", ax=ax)
# optional, but makes the labels easier to read
ax.tick_params(axis='x', labelrotation=90) 

### END SOLUTION

You should see that Convenience Stores are the largest category.

Let's keep them in the dataset for the moment, but label them.

Create a new column, `convenience`, that is `True` if the store type is `Convenience Store` and `False` otherwise. Note that `True` and `False` should be boolean values, not the strings `"True"` and `"False"`. 

In [ ]:
# your code here

### BEGIN SOLUTION
snapDf['convenience'] = snapDf.Store_Type=='Convenience Store'
### END SOLUTION

In [ ]:
# Autograding tests - do not edit
print(snapDf.convenience.mean().round(2))
print(snapDf.convenience.dtype)
assert snapDf.convenience.mean().round(2) == 0.38
assert snapDf.convenience.dtype=='bool'

If you look at the `City` field, there's some cleaning that needs to be done. You can see this by running the following cell:

In [ ]:
snapDf.City.sort_values().unique()

Replace the `City` field so that all the cities are in Title case. (Title case means the first letter of each word is capitalized, such as Los Angeles or North Hollywood.)

*Hint:* The `title()` function works the same way as `upper()` and `lower()`.

There is some other cleaning that we should do (e.g. there are sometimes two spaces between "Los" and "Angeles", but let's not worry about this here.

In [ ]:
# your code here

### BEGIN SOLUTION
snapDf.City = snapDf.City.str.title()
### END SOLUTION

In [ ]:
# Autograding tests - do not edit
print(len(snapDf.City.unique()))
assert len(snapDf.City.unique()) == 170
assert 'Los Angeles' in snapDf.City.values

At this point, we might think about doing various analyses at the city level. But let's skip that, and move on to the ZIP code and census tract levels, and practice some joins.

Let's bring in the [California EnviroScreen data](https://oehha.ca.gov/calenviroscreen/maps-data). This has both demographic and environmental justice-related data, and also the spatial boundaries of census tracts. We used it in class, so it will be in your GitHub course repository.

Load the data in to a `geopandas` `GeoDataFrame` called `esGdf`, and restrict it to the tracts in LA county.

In [ ]:
# your code here
import geopandas as gpd
esGdf = 999 # replace with your code

### BEGIN SOLUTION
# grading note: you'll need to temporarily add the CES4 shapefile to the parent directory of the student repositories for this to work
# i.e. in Assignments/
esGdf = gpd.read_file('../../CalEnviroScreen/CES4 Final Shapefile.shp')
esGdf = esGdf[esGdf.County=='Los Angeles']
### END SOLUTION

In [ ]:
# Autograding tests - do not edit
print(len(esGdf))
assert len(esGdf)==2343
assert isinstance(esGdf, gpd.GeoDataFrame)

First, let's do a tabular (non-spatial) join to ZIP code.

Create a new dataframe called `zipcodes` with one row for each ZIP code, that includes the following columns:
* `n_SNAP`: the number of SNAP outlets
* `es_percentile`: the mean EnviroScreen percentile (`CIscoreP`) for census tracts in that ZIP code 

No spatial join is needed here - ZIP is already a column in your datasets.

(In practice, we might want to weight census tracts by area or population, but don't worry about that here.)

*Hints*:
- `groupby` is your friend here
- I recommend creating two temporary dataframes (or Series) at the ZIP-code level with the number of SNAP outlets and the mean EnviroScreen score
- If you get an error that `Series object has no attribute 'join'`, you can convert that Series to a DataFrame: `pd.DataFrame(your_series_name)` 
- Then, you can join them all together
- You might need to rename the columns
- Remember to include all ZIP codes, including ones without a SNAP outlet, and replace NaNs with zeros if appropriate

In [ ]:
# your code here
zipcodes = 999

### BEGIN SOLUTION
# create tmp dataframes
tmp1 = snapDf.groupby('Zip_Code').size()
tmp1.name = 'n_SNAP'
tmp2 = esGdf.groupby('ZIP').CIscoreP.mean()
tmp2.name = 'es_percentile'

# join them together. Note that the indexes should already be the ZIP code
# do a left join from the EnviroScreen data to keep all the ZIP codes
# we could also have done an outer join in any order
zipcodes = pd.DataFrame(tmp2).join(tmp1)

# replace missing data
zipcodes.fillna({'n_SNAP':0}, inplace=True)

### END SOLUTION

In [ ]:
# Autograding tests - do not edit

print(len(zipcodes))
print(zipcodes.mean())

assert len(zipcodes) == 282
assert zipcodes.es_percentile.mean().round(1) == 26.7
assert zipcodes.n_SNAP.mean().round(1) == 29.9

Now let's do a spatial join. 

`esGdf` already has a `geometry` column and is a GeoDataFrame, but `snapDf` is a regular pandas DataFrame.

Use the `Latitude` and `Longitude` columns to add a point geometry field to `snapDf`, and turn it into a `GeoDataFrame` called `snapGdf`. Remember to specify the projection!

In [ ]:
# your code here
snapGdf = 999 # replace 999 with your solution

### BEGIN SOLUTION
snapGdf = gpd.GeoDataFrame(
    snapDf, geometry=gpd.points_from_xy(snapDf.Longitude, snapDf.Latitude, crs='EPSG:4326'))
### END SOLUTION

In [ ]:
# Autograding tests - do not edit
import numpy as np
print(snapGdf.geometry.x.min())
print(snapGdf.geometry.y.max())
print(snapGdf.crs)
assert isinstance(snapGdf, gpd.GeoDataFrame)
assert np.round(snapGdf.geometry.x.min(), 2) == -118.87
assert np.round(snapGdf.geometry.y.max(), 2) == 34.80
assert '4326' in snapGdf.crs.to_string()

Now, let's join the two GeoDataFrames together. The aim: count the number of SNAP outlets and obtain the proportion of them that are convenience stores, per census tract.

There are several ways to do it. My suggestion is as follows:
1. Add the `tract` column to `snapGdf` using a spatial join
2. Aggregate `snapGdf` by the new `tract` column using `groupby()`, to get your desired aggregates by tract
3. Join those counts to `esGdf`

Let's do these one step at a time. First, add the `tract` column to `snapGdf`. I suggest you use the `gpd.sjoin()` function and a left join. Call the new GeoDataFrame `snapGdf2`.

*Hint:* You might need to reproject!

In [ ]:
# your code here
snapGdf2 = 999 # replace 999 with your code

#### BEGIN SOLUTION
snapGdf2 = gpd.sjoin(snapGdf, esGdf.to_crs('EPSG:4326'), how="left", predicate='intersects')
# if you project to 3310, you get a slightly different answer
#snapGdf2 = gpd.sjoin(snapGdf.to_crs('EPSG:3310'), esGdf,  how="left", predicate='intersects')

#### END SOLUTION

In [ ]:
# Autograding tests - do not edit
print(len(snapGdf2))

assert len(snapGdf2) in [8502, 8501] # the answer depends on the projection
assert 'Tract' in snapGdf2.columns

That should have been a 1:many join, not the 1:1 join that you might have been expecting. (You ended up with more rows than you started with). Think about what might have caused this! 

We didn't cover in class how to drop the duplicates, but the block of code below should fix it.

In [ ]:
print('{} rows in snapGdf'.format(len(snapGdf)))
print('{} rows in snapGdf2 (after join)'.format(len(snapGdf2)))

# you can also see that the ObjectId column (which identifies the SNAP providers) 
# is unique before the join, but not after
print(snapGdf.ObjectId.is_unique)  # is there one row per SNAP outlet before the join?
print(snapGdf2.ObjectId.is_unique) # is there one row per SNAP outlet after the join?

# drop the duplicates by just taking the first ObjectId that joins to each tract
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
snapGdf2.drop_duplicates(subset='ObjectId', keep='first', inplace=True)
print('{} rows in snapGdf2 (after dropping duplicates)'.format(len(snapGdf2)))

Now, aggregate snapGdf2 by the new `tract` column to create a dataframe with (i) the number of all SNAP outlets in that tract, and (ii) the proportion of SNAP outlets that are convenience stores in that tract. 

Call those columns `n_snap` and `frc_convenience`, and the dataframe `tract_counts`.

In [ ]:
tract_counts = 999 # your code here

### BEGIN SOLUTION
# here, we get the mean of `convenience` and the number
# we could also have aggregated the count and the mean separately, and done another join
tract_counts = snapGdf2.groupby('Tract').convenience.agg(['count', 'mean'])
tract_counts.rename(columns={'count':'n_snap', 'mean':'frc_convenience'}, inplace=True)
### END SOLUTION


In [ ]:
# Autograding tests - do not edit
print(tract_counts.mean())
assert np.round(tract_counts.n_snap.mean(),1)==4.3 
assert np.round(tract_counts.frc_convenience.mean(),2) in [0.42, 42.96]


Final step in the join process! Let's join `tract_counts` back to `esGdf` to create a new geodataframe called `joinedGdf`. 

This should be a left join (so you don't leave out any census tracts) on the `Tract` column.

Remember to fill in any missing data for `n_snap` with zeros. (Should you also fill in the NaNs for `frc_convenience`? Probably not - if there aren't any SNAP outlets, we actually don't have any information on the fraction, and so it's more appropriate to leave it as NaN.

In [ ]:
joinedGdf = 999 # your answer here

### BEGIN SOLUTION
joinedGdf = esGdf.set_index('Tract').join(tract_counts, how='left')
joinedGdf.fillna({'n_snap':0} , inplace=True)
### END SOLUTION

In [ ]:
# Autograding tests - do not edit

print(len(joinedGdf))
print(joinedGdf.n_snap.sum(), joinedGdf.frc_convenience.sum())
assert len(joinedGdf) == len(esGdf)
assert tract_counts.n_snap.sum() == joinedGdf.n_snap.sum()
assert joinedGdf.n_snap.count() == len(joinedGdf)

Now plot the `n_snap` and `frc_convenience` columns (2 separate maps). Use the examples from class. At a minimum, your maps should have:
* a basemap (e.g. from contextily)
* a legend or colorbar
* a title

If you can figure it out, you might want to drop Catalina Island to focus on mainland LA County. (Hint: the `ax.set_ylim()` function is useful here.)

Your code will be shorter, more elegant and easier to understand if you use a loop! But it's fine if you don't do it that way.

In [ ]:
# your code here
import contextily as ctx
import matplotlib.pyplot as plt

### BEGIN SOLUTION
# since we are doing 2 plots that are pretty much identical, we can use a loop!
# this creates axes as a list of axes objects
for colname in ['n_snap','frc_convenience']:
    fig, ax = plt.subplots(figsize=(10,5))
    joinedGdf.to_crs('EPSG:3857').plot(colname, cmap='viridis', legend=True, ax = ax, alpha=0.4)

    # drop Catalina
    ax.set_ylim([3.98e6, 4.14e6])
    ctx.add_basemap(ax, zoom=12, source=ctx.providers.OpenStreetMap.Mapnik)

    # and we really don't need the axis ticks and labels, so we set them to an empty list
    ax.set_xticks([])
    ax.set_yticks([])

    # here, we can do stuff that is a bit different across plots
    if colname == 'n_snap':
        titletext = 'Number of SNAP outlets' 
    else:
        titletext = 'Fraction of convenience stores' 
        
    ax.set_title(titletext, fontsize=12)

### END SOLUTION

Reflect on this assignment. What did you find most challenging? What problems did you encounter? How might you have gone about it differently the next time? (Write a few bullet points in a markdown cell.)

To help me calibrate future assignments, please also indicate about how long it took you to complete.

# Challenge Problem
Remember, you need to do at least two of these challenge problems this quarter.

We mapped the number of grocery stores, but didn't say anything directly about food deserts. In the challenge, take this analysis further. My suggestion:
* Normalize your number of outlets (e.g. by population) and plot these data
* Plot the normalized number of outlets (both including and excluding convenience stores) against race, income, and other variables from EnviroScreen
* Think about boundary issues created by the artefacts of census geography. Create a measure of the number of outlets within (say) 2km of a census tract boundary, even if they do not intersect that tract
* Briefly write a few sentences that intepret your results

If you want to go further, please do!







